## HAMOYE STAGE B- Oluyemisi Yesufu

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/gdrive')
df1 = pd.read_excel('/content/gdrive/My Drive/zindiWomen/Hamoye_new/ENB2012_data.xlsx')
df2 = pd.read_csv('/content/gdrive/My Drive/zindiWomen/Hamoye_new/energydata_complete.csv')

Mounted at /content/gdrive


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso,Ridge



In [3]:
df2.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


### Question 17

From the dataset, fit a linear model on the relationship between the temperature in the living room in Celsius (x = T2) and the temperature outside the building (y = T6). What is the Root Mean Squared error in three D.P?

In [12]:
LR = LinearRegression()
scaler = MinMaxScaler()
##get he features and target variable
features_df = np.array(df2['T2']).reshape(-1,1)
target = df2['T6']
##prepare the training and testing dataset
x_train, x_test, y_train, y_test = train_test_split(features_df, target,
test_size=0.3, random_state=1)
#fit the model to the training dataset
LR.fit(x_train, y_train)
#obtain predictions
predicted_values = LR.predict(x_test)
rmse= np.sqrt(mean_squared_error(y_test,predicted_values))
round(rmse,3)

3.612

The root mean squared error for the model is 3.612

### Question 18


Remove the following columns: [“date”, “lights”]. The target variable is “Appliances”. Use a 70-30 train-test set split with a  random state of 42 (for reproducibility). Normalize the dataset using the MinMaxScaler (Hint: Use the MinMaxScaler fit_transform and transform methods on the train and test set respectively). Run a multiple linear regression using the training set. Answer the following questions:

What is the Mean Absolute Error (in three decimal places) for the  training set?

In [19]:
features_df = df2.drop(columns=['date', 'lights','Appliances'])
target=df2['Appliances']
normalised_df = pd.DataFrame(scaler.fit_transform(features_df), columns=features_df.columns)
df = pd.DataFrame(scaler.fit_transform(features_df), columns=features_df.columns)
x_train, x_test, y_train, y_test = train_test_split(df, target,
test_size=0.3, random_state=42)
LR.fit(x_train, y_train)
predicted_values = LR.predict(x_train)
mae_train = mean_absolute_error(y_train, predicted_values)
round(mae_train, 3) #prints 0.063

53.742

The mean absolute error is **53.742**

### Question 19

What is the Root Mean Squared Error (in three decimal places) for the training set?

In [17]:
predicted_values = LR.predict(x_train)
rmse = np.sqrt(mean_squared_error(y_train, predicted_values))
round(rmse, 3)

95.216

The Root mean squared error is **95.216**

### Question 20

What is the Mean Absolute Error (in three decimal places) for test set?

In [18]:
predicted_values = LR.predict(x_test)
mae_test = mean_absolute_error(y_test, predicted_values)
round(mae_test, 3) #prints 0.063

53.643

The mean absolute error for the test set is **53.643**

### Question 21

What is the Root Mean Squared Error (in three decimal places) for test set?

In [21]:
predicted_values = LR.predict(x_test)
rmse_test = np.sqrt(mean_squared_error(y_test, predicted_values))
round(rmse_test, 3)

93.64

The root mean square error for the test set is **93.64**

### Question 22

Did the Model above overfit to the training set

No it did not because it performes better on the test set.

### Question 23

Train a ridge regression model with default parameters. Is there any change to the root mean squared error (RMSE) when evaluated on the test set?

In [22]:
ridge_reg = Ridge()
ridge_reg.fit(x_train,y_train)
predicted_values = ridge_reg.predict(x_test)
rmse_ridge = np.sqrt(mean_squared_error(y_test, predicted_values))
round(rmse_ridge, 3)

93.716

### Question 24


Train a lasso regression model with default value and obtain the new feature weights with it. How many of the features have non-zero feature weights?

In [23]:
def get_weights_df(model, feat, col_name):
    d = model.coef_
    feat = pd.DataFrame(feat)
    m=[]
    for i in d:
        if type(i)==np.float64:
            m.append(i)
        else:
            for j in i:
                m.append(j)
  #this function returns the weight of every feature
    weights = pd.Series(m, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['Features', col_name]
    weights_df[col_name].round(3)

    return weights_df

In [27]:
lasso_reg = Lasso()
lasso_reg.fit(x_train,y_train)
lasso_weights_df = get_weights_df(lasso_reg, x_train, 'Lasso_weight')

lasso_weights_df


,Features,Lasso_weight
0,RH_out,-52.139924
1,RH_8,-11.499414
2,T1,0.000000
3,Tdewpoint,0.000000
4,Visibility,0.000000
5,Press_mm_hg,-0.000000
6,T_out,0.000000
7,RH_9,-0.000000
8,T9,-0.000000
9,T8,0.000000


Answer: Only four columns have non zero feature weights

### Question 25

What is the new RMSE with the Lasso Regression on the test set?

In [29]:
predicted_values = lasso_reg.predict(x_test)
rmse_lasso = np.sqrt(mean_squared_error(y_test, predicted_values))
round(rmse_lasso, 3)

99.813

The root mean squared error is **99.813**